In [49]:
from PIL import Image
import math

def remove_circle(img_name):

    original_im = Image.open('../../ISIC-2017_Training_Data/ISIC_' + img_name + '.jpg')
    rgb_original_im = original_im.convert('RGB')
    
    width, height = original_im.size
    
    radius = int(max(width, height) / math.sqrt(2))
    
    percentage_black = 1
    center_x = width/2
    center_y = height/2
    
    excluded_pixels = {}
    output = None
    
    while radius > 0 and percentage_black > 0.65:
        output = original_im.copy() #Image.new( 'RGB', (width,height), "black") # create a new black image
        output_pixels = output.load() # create the pixel map
        
        circle_upper = {}
        circle_lower = {}
        
        for x in range(-radius,radius):
            y_upper = center_y + int(math.sqrt(radius*radius - x*x))
            y_lower = center_y - int(math.sqrt(radius*radius - x*x))
            x_val = center_x + x

            if x_val > 0 and x_val < width and y_upper > 0 and y_upper < height and y_lower > 0 and y_lower < height:               
                circle_upper[x_val] = y_upper
                circle_lower[x_val] = y_lower
        outside_pixels = []
        excluded_pixels = {}
        dark_ctr = 0.0
        for y in range(height):
            for x in range(width):
                try:
                    if x < (center_x-radius) or x > (center_x+radius-1) or y > circle_upper[x] or y < circle_lower[x]:
                        output_pixels[x,y] = (255, 255, 255)
                        r, g, b = rgb_original_im.getpixel((x,y))
                        avg = long((r + b + g))/3.0
                        outside_pixels.append(avg)
                        
                        if x in excluded_pixels:
                            excluded_pixels[x].add(y)
                        else:
                            excluded_pixels[x] = set()
                            excluded_pixels[x].add(y)
                        
                        if avg < 30:
                            dark_ctr += 1.0
                except KeyError:
                    continue
                    
        var = len(outside_pixels)
        if not(var == 0):            
            percentage_black = dark_ctr/long(len(outside_pixels))
            output.save("./circle_tests/" + img_name + str(radius) + ".png")
        radius = radius - 5
    return excluded_pixels

In [50]:
from skimage.transform import rescale
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

%matplotlib inline

def group_clusters(img_name):

    cluster_im = Image.open('../../cmeans_noblock/ISIC_' + img_name + '_visual.gif')
    original_im = Image.open('../../ISIC-2017_Training_Data/ISIC_' + img_name + '.jpg')
    rgb_cluster_im = cluster_im.convert('RGB')
    rgb_original_im = original_im.convert('RGB')

    scale_factor = float(cluster_im.size[0]) / float(original_im.size[0])
    circle_pixels = remove_circle(img_name)
    forbidden_clusters = set()

    clusters = {}

    #collect pixel values for each cluster
    width, height = cluster_im.size
    for y in range(height):
        for x in range(width):
            r, g, b = rgb_cluster_im.getpixel((x, y))
            cluster_id = str(r) + '-' + str(g) + '-' + str(b)
            
            if x in circle_pixels and y in circle_pixels[x]:
                forbidden_clusters.add(cluster_id)
            
            if cluster_id not in forbidden_clusters:
                original_rgb = rgb_original_im.getpixel((x/scale_factor, y/scale_factor)) #.3258 to deal with scale difference

                if cluster_id in clusters:
                    clusters[cluster_id].append(original_rgb)
                else:
                    clusters[cluster_id] = [original_rgb]

    cluster_averages = {}

    #get average colour of clusters
    for cluster_id in clusters:
        if cluster_id not in forbidden_clusters:
            r, g, b = 0, 0, 0
            count = 0
            for pixel in clusters[cluster_id]:
                r += pixel[0]
                g += pixel[1]
                b += pixel[2]
                count += 1
            cluster_averages[cluster_id] = ((r/count), (g/count), (b/count))

    #Create Data for clustering
    list_of_pixels = []
    list_of_ids = {}
    counter = 0
    for cluster_id in cluster_averages:
        list_of_ids[cluster_id] = counter
        counter += 1
        list_of_pixels.append(list(cluster_averages[cluster_id]))

    cvecs = np.array(list_of_pixels)

    #Cluster using k-means
    kmns = KMeans(n_clusters=2, random_state=1729)
    kmns.fit(cvecs)
    
    #Find darker of centroids
    cluster1 = kmns.cluster_centers_[0]
    cluster2 = kmns.cluster_centers_[1]
    
    avg1 = (cluster1[0] + cluster1[1] + cluster1[2])/3
    avg2 = (cluster2[0] + cluster2[1] + cluster2[2])/3

    #Generate output using clustered labels

    output = Image.new( 'RGB', (width,height), "black") # create a new black image
    output_pixels = output.load() # create the pixel map

    width, height = cluster_im.size
    for y in range(height):
        for x in range(width):
            r, g, b = rgb_cluster_im.getpixel((x, y))
            cluster_id = str(r) + '-' + str(g) + '-' + str(b)
            if cluster_id in list_of_ids:
                index = list_of_ids[cluster_id]
                if kmns.labels_[index] and avg2 < avg1:
                    output_pixels[x,y] = (255, 255, 255) # set the colour accordingly
                elif not kmns.labels_[index] and avg1 < avg2:
                    output_pixels[x,y] = (255, 255, 255)

    plt.imshow(output)
    output.save("./grouped_clusters/" + img_name + ".png")

In [51]:
# #for photo_number in range(0, 15295):    
# for photo_number in range(0, 45):    
#     try:
#         id = str(photo_number)
#         new_id = ''

#         num_digits = len(id)

#         if num_digits < 7:
#             difference = 7 - num_digits

#             for digit in range(0, difference):
#                 new_id = new_id + '0'
#             id = new_id + id

#         result = group_clusters(id)
        
#     except IOError:
#         continue


group_clusters('0000006')
group_clusters('0000011')
group_clusters('0000030')
group_clusters('0000036')
group_clusters('0000045')

group_clusters('0000169')





##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########
##########

KeyboardInterrupt: 